In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--lang=es')
# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://security.ubuntu.com/ub

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from selenium import webdriver
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Data Analyst/Spiderum

/content/drive/MyDrive/Data Analyst/Spiderum


In [ ]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [ ]:
spiderum = pd.DataFrame(columns = ['Title','Author', 'Category', 'Upvote', 'View', 'Comment', 'Reading_time', 'Posting_time', 'Len_content', 'Len_comment'])
for i in range(1461, 1500): #1458
    print(i)
    link_page = 'https://spiderum.com/s/all/new?page=' + str(i)
    try:
        req = requests.get(link_page)
        page_obj = BeautifulSoup(req.text, 'html.parser')
    except:
        print('There is error in num page {} about link'.format(i))

    list_url = []
    list_len_content = []
    list_len_comment = []
    list_title = []
    list_author = []
    list_category = []
    list_upvote = []
    list_view = []
    list_comment = []
    list_reading_time = []
    list_posting_time = []

    # Extract urls, titles
    try:
        for link in page_obj.find_all('a', {'rel': 'canonical'}):
            list_url.append('https://spiderum.com' + link.attrs['href'])
            list_title.append(link.text.replace('\n', '').replace('\t', ''))
    except:
        print('There is error in num page {} about urls and titles'.format(i))
  
    # Extract authors
    try:
        for link in page_obj.find_all('a', {'class': 'username'}):
            list_author.append(link.text.replace('\n', '').replace('\t', ''))
    except:
        print('There is error in num page {} about authors'.format(i))

    # Extract categories
    try:
        for link in page_obj.find_all('a', {'class': 'category-name'}):
            list_category.append(link.text)
    except:
        print('There is error in num page {} about category'.format(i))

    # Extract upvotes
    try:
        for link in page_obj.find_all('span', {'class': 'vote-count'}):
            list_upvote.append(link.text)
    except:
        print('There is error in num page {} about upvote'.format(i))

    # Extract comments
    try:
        for link in page_obj.select('a.comment > span.text'):
            list_comment.append(link.text)
    except:
        print('There is error in num page {} about comment'.format(i))

    # Extract reading time and posting time
    try:
        for index, link in enumerate(page_obj.select('div.created > span')):
            if index % 2 == 0:
                list_posting_time.append(link.text)
            else:
                list_reading_time.append(link.text)
    except:
        print('There is error in num page {} about reading and posting time'.format(i))
        
    # Extract contents, comments and views
    for index, article_url in enumerate(list_url):
        try:
            driver.get(article_url)
        except:
            print('There is error in article title "{}" in page {} about loading'.format(list_title[index], i))
            list_view.append(0)
            list_len_content.append(0)
            list_len_comment.append(0)
            continue

        # views
        try:
            view = driver.find_element_by_xpath('//div[@class="views"]').text.replace(' lượt xem', '')
        except:
            view = 0
        list_view.append(view)
        
        # len content
        try:
            len_content = len(driver.find_element_by_xpath('//div[@class="content"]').text)
            list_len_content.append(len_content)
        except:
            print('There is error in article title "{}" in page {} about content'.format(list_title[index], i))
        
        # len comments
        try:
            count = 0
            while check_exists_by_xpath('//a[contains(text(), "Xem thêm bình luận")]'):
                driver.find_element_by_xpath('//a[contains(text(), "Xem thêm bình luận")]').click()
                time.sleep(0.5)
                count += 1
                if count == 50:
                    print('There is error in article title "{}" in page {} about len comment'.format(list_title[index], i))
                    break

            len_comment = len(driver.find_element_by_xpath('//div[@class="comment-box"]').text)
            list_len_comment.append(len_comment)
        except:
            list_len_comment.append(0)
            print('There is error in article title "{}" in page {} about len comment'.format(list_title[index], i))
        
    twenty_info =  list(zip(list_title, list_author, list_category, list_upvote, list_view, list_comment, list_reading_time, list_posting_time, list_len_content, list_len_comment))

    for info in twenty_info:
        spiderum.loc[len(spiderum)] = info

In [ ]:
spiderum.to_csv('./spiderum 1460-end.csv', index=False)

## II. Concatenate files

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Data Analyst/Spiderum/data

/content/drive/MyDrive/Data Analyst/Spiderum/data


In [10]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np

files = os.listdir()

In [11]:
files = sorted(files)[:2] + sorted(files)[9:] + sorted(files)[2:9]
files

In [13]:
# Concatenate many single files into one
data = pd.read_csv('./' + files[0])
for file in files:
    other_data = pd.read_csv(file)
    data = pd.concat([data, other_data], ignore_index=True)

In [14]:
data.tail()

,Title,Author,Category,Upvote,View,Comment,Reading_time,Posting_time,Len_content,Len_comment
31419,Bồ Đào Nha: Cho những ngày điệu Fado không còn...,Đức Đen,Tin tức,12,2786,6,· 2 phút đọc,12 tháng 7 2016,2196,1183
31420,Sử dụng túi ny-lông – Cấm hay không cấm?!,caudocthan,Tin tức,15,2616,13,· 4 phút đọc,12 tháng 7 2016,4920,2917
31421,Đàn ông và Thuốc lá,joel15,Quan điểm - Tranh luận,14,4917,11,· 2 phút đọc,12 tháng 7 2016,2497,1197
31422,Như thế nào là sếp tốt?,Tiểu AnhAnh,Chuyện trò - Tâm sự,15,2290,5,· 4 phút đọc,12 tháng 7 2016,4113,639
31423,Đàn bà rất khoái hi sinh?,ruatobu,Chuyện trò - Tâm sự,9,2266,3,· 1 phút đọc,12 tháng 7 2016,1591,1487


In [ ]:
data.to_csv('./spiderum.csv', index=False)